In [14]:
import os

import numpy as np
import pandas as pd

from fma_code import utils
from utils import DS_PATH

%matplotlib inline

pd.options.display.max_rows = None
pd.options.display.max_columns = None
pd.options.display.max_colwidth = None

In [15]:
TRACKS_PATH = os.path.join(DS_PATH, 'tracks.csv')
tracks = utils.load(TRACKS_PATH)
tracks.head(10)

album                                                          \
         comments        date_created date_released engineer favorites id   
track_id                                                                    
2               0 2008-11-26 01:44:45    2009-01-05      NaN         4  1   
3               0 2008-11-26 01:44:45    2009-01-05      NaN         4  1   
5               0 2008-11-26 01:44:45    2009-01-05      NaN         4  1   
10              0 2008-11-26 01:45:08    2008-02-06      NaN         4  6   
20              0 2008-11-26 01:45:05    2009-01-06      NaN         2  4   
26              0 2008-11-26 01:45:05    2009-01-06      NaN         2  4   
30              0 2008-11-26 01:45:05    2009-01-06      NaN         2  4   
46              0 2008-11-26 01:45:05    2009-01-06      NaN         2  4   
48              0 2008-11-26 01:45:05    2009-01-06      NaN         2  4   
134             0 2008-11-26 01:44:45    2009-01-05      NaN         4  1   

                                                                           \
                                        information listens producer tags   
track_id                                                                    
2                                           <p></p>    6073      NaN   []   
3                                           <p></p>    6073      NaN   []   
5                                           <p></p>    6073      NaN   []   
10                                              NaN   47632      NaN   []   
20        <p> "spiritual songs" from Nicky Cook</p>    2710      NaN   []   
26        <p> "spiritual songs" from Nicky Cook</p>    2710      NaN   []   
30        <p> "spiritual songs" from Nicky Cook</p>    2710      NaN   []   
46        <p> "spiritual songs" from Nicky Cook</p>    2710      NaN   []   
48        <p> "spiritual songs" from Nicky Cook</p>    2710      NaN   []   
134                                         <p></p>    6073      NaN   []   

                                                        artist  \
                         title tracks   type active_year_begin   
track_id                                                         
2         AWOL - A Way Of Life      7  Album        2006-01-01   
3         AWOL - A Way Of Life      7  Album        2006-01-01   
5         AWOL - A Way Of Life      7  Album        2006-01-01   
10           Constant Hitmaker      2  Album               NaT   
20                       Niris     13  Album        1990-01-01   
26                       Niris     13  Album        1990-01-01   
30                       Niris     13  Album        1990-01-01   
46                       Niris     13  Album        1990-01-01   
48                       Niris     13  Album        1990-01-01   
134       AWOL - A Way Of Life      7  Album        2006-01-01   

                          \
         active_year_end   
track_id                   
2                    NaT   
3                    NaT   
5                    NaT   
10                   NaT   
20            2011-01-01   
26            2011-01-01   
30            2011-01-01   
46            2011-01-01   
48            2011-01-01   
134                  NaT   

                                                                \
                                             associated_labels   
track_id                                                         
2                                                          NaN   
3                                                          NaN   
5                                                          NaN   
10        Mexican Summer, Richie Records, Woodsist, Skulltones   
20                                                         NaN   
26                                                         NaN   
30                                                         NaN   
46                                                         NaN   
48                                                        

In [16]:
tracks = pd.merge(tracks.get('set'), tracks.get('track'), left_index=True, right_index=True)
tracks.head(10)

,split,subset,bit_rate,comments,composer,date_created,date_recorded,duration,favorites,genre_top,genres,genres_all,information,interest,language_code,license,listens,lyricist,number,publisher,tags,title
track_id,,,,,,,,,,,,,,,,,,,,,,
2,training,small,256000,0,NaN,2008-11-26 01:48:12,2008-11-26,168,2,Hip-Hop,[21],[21],NaN,4656,en,Attribution-NonCommercial-ShareAlike 3.0 International,1293,NaN,3,NaN,[],Food
3,training,medium,256000,0,NaN,2008-11-26 01:48:14,2008-11-26,237,1,Hip-Hop,[21],[21],NaN,1470,en,Attribution-NonCommercial-ShareAlike 3.0 International,514,NaN,4,NaN,[],Electric Ave
5,training,small,256000,0,NaN,2008-11-26 01:48:20,2008-11-26,206,6,Hip-Hop,[21],[21],NaN,1933,en,Attribution-NonCommercial-ShareAlike 3.0 International,1151,NaN,6,NaN,[],This World
10,training,small,192000,0,Kurt Vile,2008-11-25 17:49:06,2008-11-26,161,178,Pop,[10],[10],NaN,54881,en,Attribution-NonCommercial-NoDerivatives (aka Music Sharing) 3.0 International,50135,NaN,1,NaN,[],Freeway
20,training,large,256000,0,NaN,2008-11-26 01:48:56,2008-01-01,311,0,NaN,"[76, 103]","[17, 10, 76, 103]",NaN,978,en,Attribution-NonCommercial-NoDerivatives (aka Music Sharing) 3.0 International,361,NaN,3,NaN,[],Spiritual Level
26,training,large,256000,0,NaN,2008-11-26 01:49:05,2008-01-01,181,0,NaN,"[76, 103]","[17, 10, 76, 103]",NaN,1060,en,Attribution-NonCommercial-NoDerivatives (aka Music Sharing) 3.0 International,193,NaN,4,NaN,[],Where is your Love?
30,training,large,256000,0,NaN,2008-11-26 01:49:11,2008-01-01,174,0,NaN,"[76, 103]","[17, 10, 76, 103]",NaN,718,en,Attribution-NonCommercial-NoDerivatives (aka Music Sharing) 3.0 International,612,NaN,5,NaN,[],Too Happy
46,training,large,256000,0,NaN,2008-11-26 01:49:53,2008-01-01,104,0,NaN,"[76, 103]","[17, 10, 76, 103]",NaN,252,en,Attribution-NonCommercial-NoDerivatives (aka Music Sharing) 3.0 International,171,NaN,8,NaN,[],Yosemite
48,training,large,256000,0,NaN,2008-11-26 01:49:56,2008-01-01,205,0,NaN,"[76, 103]","[17, 10, 76, 103]",NaN,247,en,Attribution-NonCommercial-NoDerivatives (aka Music Sharing) 3.0 International,173,NaN,9,NaN,[],Light of Light


In [17]:
tracks = tracks[['subset', 'genre_top', 'genres', 'genres_all']]
tracks.head(5)

,subset,genre_top,genres,genres_all
track_id,,,,
2,small,Hip-Hop,[21],[21]
3,medium,Hip-Hop,[21],[21]
5,small,Hip-Hop,[21],[21]
10,small,Pop,[10],[10]
20,large,NaN,"[76, 103]","[17, 10, 76, 103]"


In [18]:
tracks = tracks[tracks['subset'] <= 'small']
tracks.shape

(8000, 4)

In [3]:
AUDIO_PATH = os.path.join(os.getcwd(), 'fma_medium')

viable_tracks_ids = []
for dir_el in os.listdir(AUDIO_PATH):
    if dir_el in ('checksums', 'README.txt'): continue
    for track_filename in os.listdir(os.path.join(AUDIO_PATH, dir_el)):
        while track_filename.startswith('0'):
            track_filename = track_filename[1:]
        viable_tracks_ids.append(int(track_filename.removesuffix('.mp3')))
viable_tracks_ids

[2,
 3,
 5,
 10,
 134,
 136,
 139,
 140,
 141,
 148,
 181,
 182,
 190,
 193,
 194,
 197,
 198,
 200,
 203,
 207,
 208,
 209,
 210,
 211,
 212,
 213,
 237,
 247,
 249,
 255,
 256,
 257,
 258,
 341,
 343,
 352,
 368,
 369,
 398,
 399,
 400,
 405,
 414,
 424,
 425,
 458,
 459,
 512,
 534,
 535,
 536,
 537,
 540,
 546,
 550,
 564,
 573,
 583,
 584,
 585,
 590,
 591,
 592,
 593,
 602,
 603,
 605,
 608,
 615,
 619,
 620,
 621,
 625,
 629,
 631,
 633,
 664,
 665,
 666,
 667,
 668,
 669,
 672,
 673,
 674,
 676,
 677,
 679,
 680,
 681,
 684,
 686,
 690,
 694,
 695,
 204,
 397,
 574,
 635,
 696,
 737,
 758,
 814,
 861,
 698,
 701,
 702,
 703,
 704,
 705,
 706,
 707,
 708,
 709,
 714,
 715,
 716,
 717,
 718,
 720,
 734,
 735,
 736,
 738,
 739,
 740,
 741,
 742,
 743,
 744,
 745,
 747,
 748,
 749,
 750,
 751,
 752,
 753,
 754,
 755,
 756,
 757,
 759,
 760,
 761,
 762,
 763,
 764,
 765,
 766,
 767,
 768,
 769,
 770,
 771,
 772,
 773,
 774,
 775,
 776,
 777,
 819,
 820,
 821,
 822,
 823,
 824,
 825,

In [4]:
assert set(viable_tracks_ids).intersection(tracks.index) == set(tracks.index).intersection(viable_tracks_ids)
viable_tracks_ids = list(set(viable_tracks_ids).intersection(tracks.index))
print(tracks.shape)
tracks = tracks.loc[viable_tracks_ids]
tracks.shape

(22942, 12)


(22942, 12)

In [5]:
assert all(idx in viable_tracks_ids for idx in tracks.index)

In [6]:
tracks.reset_index(inplace=True)
tracks.sample(15)

,track_id,Chip Music,Electronic,Experimental,Folk,Garage,Hip-Hop,Indie-Rock,Instrumental,International,Pop,Punk,Rock
19661,113940,0,0,0,0,1,0,0,0,0,0,0,0
18103,105716,0,0,0,1,0,0,0,0,0,0,0,0
20668,119941,0,0,0,0,0,1,0,0,0,0,0,0
13014,67239,0,0,0,0,0,0,0,0,0,0,0,1
19004,110627,0,0,1,0,0,0,0,0,0,0,0,0
7210,28259,0,0,0,0,0,0,1,0,0,0,0,0
7736,32691,0,0,0,0,0,1,0,0,0,0,0,0
3055,10685,0,0,0,1,0,0,0,0,0,0,0,0
20275,117237,0,0,0,0,1,0,0,0,0,0,0,0
2963,10382,0,0,0,1,0,0,0,0,0,0,0,0


In [7]:
X = tracks[['track_id']]
y = tracks.drop('track_id', axis=1)
X.shape, y.shape

((22942, 1), (22942, 12))

In [8]:
from sklearn.model_selection import train_test_split

train_tracks, val_tracks, train_labels, val_labels = train_test_split(X, y, test_size=0.3, stratify=y)
val_tracks, test_tracks, val_labels, test_labels = train_test_split(val_tracks, val_labels, test_size=0.33, stratify=val_labels)
train_tracks.shape, train_labels.shape, val_tracks.shape, val_labels.shape, test_tracks.shape, test_labels.shape

((16059, 1), (16059, 12), (4611, 1), (4611, 12), (2272, 1), (2272, 12))

In [9]:
train_labels.columns[(train_labels == 0).all()]

Index([], dtype='object')

In [10]:
val_labels.columns[(val_labels == 0).all()]

Index([], dtype='object')

In [11]:
test_labels.columns[(test_labels == 0).all()]

Index([], dtype='object')

In [12]:
set(y.columns) - set(test_labels.columns[(test_labels == 0).all()])

{'Chip Music',
 'Electronic',
 'Experimental',
 'Folk',
 'Garage',
 'Hip-Hop',
 'Indie-Rock',
 'Instrumental',
 'International',
 'Pop',
 'Punk',
 'Rock'}

In [13]:
pd.merge(train_tracks, train_labels, left_index=True, right_index=True).sample(10)

,track_id,Chip Music,Electronic,Experimental,Folk,Garage,Hip-Hop,Indie-Rock,Instrumental,International,Pop,Punk,Rock
11202,56558,0,0,0,0,0,0,0,0,0,0,0,1
11843,60881,0,0,0,0,0,0,0,0,0,0,1,0
4492,145895,0,0,1,0,0,0,0,0,0,0,0,0
20787,120448,0,1,0,0,0,0,0,0,0,0,0,0
21373,123761,0,0,0,0,0,0,0,1,0,0,0,0
19630,113559,0,0,1,0,0,0,0,0,0,0,0,0
1880,137551,0,0,1,0,0,0,0,0,0,0,0,0
8910,41938,0,1,0,0,0,0,0,0,0,0,0,0
14318,74668,0,0,0,0,0,0,0,0,0,0,0,1
2979,10441,0,0,0,1,0,0,0,0,0,0,0,0


In [14]:
SPLIT_DURATION = 3.0
REPOPULATE = True

In [15]:
label_columns = tracks.drop('track_id', axis=1).columns
label_columns

Index(['Chip Music', 'Electronic', 'Experimental', 'Folk', 'Garage', 'Hip-Hop',
       'Indie-Rock', 'Instrumental', 'International', 'Pop', 'Punk', 'Rock'],
      dtype='object')

In [16]:
import multiprocessing as mp
from tqdm.notebook import tqdm
from spectrograms import load_audio_file, generate_track_melspectrograms, pickle_spectrograms_set
tqdm.pandas()


def create_spectrograms_for_set(set_tracks_ids: pd.Index, split_duration: float = SPLIT_DURATION, max_shape: tuple[int, int] = (0, 0)) -> tuple[np.ndarray, np.ndarray]:
    X = []
    y = []
    set_tracks = tracks.loc[set_tracks_ids, 'track_id']
    
    with mp.Pool(os.cpu_count() - 1) as pool:
        for idx, track_id in tqdm(set_tracks.items(), total=len(set_tracks)):
            audio, sampling_rate = load_audio_file(track_id)
            if not isinstance(audio, np.ndarray):
                print(f'{track_id=} cannot be loaded')
                continue
            
            track_spectrograms = pool.apply(generate_track_melspectrograms, args=(audio, sampling_rate, split_duration,))

            if not isinstance(track_spectrograms, np.ndarray): continue
            
            for track_spectrogram in track_spectrograms:
                if track_spectrogram.shape > max_shape:
                    max_shape = track_spectrogram.shape
                X.append(track_spectrogram)
            
            track_genre = tracks.loc[idx, label_columns]
            for _ in range(len(track_spectrograms)):
                y.append(track_genre)
        
        for idx, track_spectrogram in enumerate(X):
            if track_spectrogram.shape == max_shape: continue
            X[idx] = np.resize(track_spectrogram, max_shape)
            
    return np.array(X, dtype=np.float16), np.array(y, dtype=np.float16), max_shape

In [17]:
x_train, y_train, max_shape = create_spectrograms_for_set(train_tracks.index)
x_train.shape, y_train.shape

  0%|          | 0/16059 [00:00<?, ?it/s]

[src/libmpg123/parse.c:do_readahead():1099] warning: Cannot read next header, a one-frame stream? Duh...
/mnt/WD-shared/PJATK/PracaMagisterska/predictions_server/model_testing/data/FMA/spectrograms.py:26: UserWarning: PySoundFile failed. Trying audioread instead.
  return librosa.load(file_path, sr=SAMPLING_RATE, mono=True)


track_id=65753 cannot be loaded


[src/libmpg123/layer3.c:INT123_do_layer3():1801] error: dequantization failed!
[src/libmpg123/parse.c:do_readahead():1099] warning: Cannot read next header, a one-frame stream? Duh...
/mnt/WD-shared/PJATK/PracaMagisterska/predictions_server/model_testing/data/FMA/spectrograms.py:26: UserWarning: PySoundFile failed. Trying audioread instead.
  return librosa.load(file_path, sr=SAMPLING_RATE, mono=True)


track_id=98558 cannot be loaded


...

track_id=5574 cannot be loaded


[src/libmpg123/layer3.c:INT123_do_layer3():1801] error: dequantization failed!
[src/libmpg123/parse.c:do_readahead():1099] warning: Cannot read next header, a one-frame stream? Duh...
/mnt/WD-shared/PJATK/PracaMagisterska/predictions_server/model_testing/data/FMA/spectrograms.py:26: UserWarning: PySoundFile failed. Trying audioread instead.
  return librosa.load(file_path, sr=SAMPLING_RATE, mono=True)


track_id=1486 cannot be loaded


[src/libmpg123/layer3.c:INT123_do_layer3():1801] error: dequantization failed!


((167117, 128, 130), (167117, 12))

In [18]:
pickle_spectrograms_set(x_train, y_train, 'train')

In [19]:
del x_train, y_train

In [20]:
x_val, y_val, _ = create_spectrograms_for_set(val_tracks.index, max_shape=max_shape)
x_val.shape, y_val.shape

  0%|          | 0/4611 [00:00<?, ?it/s]

Note: Illegal Audio-MPEG-Header 0x00000000 at offset 33361.
Note: Trying to resync...
Note: Skipped 1024 bytes in input.
[src/libmpg123/parse.c:wetwork():1365] error: Giving up resync after 1024 bytes - your stream is not nice... (maybe increasing resync limit could help).
/mnt/WD-shared/PJATK/PracaMagisterska/predictions_server/model_testing/data/FMA/spectrograms.py:26: UserWarning: PySoundFile failed. Trying audioread instead.
  return librosa.load(file_path, sr=SAMPLING_RATE, mono=True)
[src/libmpg123/layer3.c:INT123_do_layer3():1841] error: dequantization failed!
[src/libmpg123/parse.c:do_readahead():1099] warning: Cannot read next header, a one-frame stream? Duh...
/mnt/WD-shared/PJATK/PracaMagisterska/predictions_server/model_testing/data/FMA/spectrograms.py:26: UserWarning: PySoundFile failed. Trying audioread instead.
  return librosa.load(file_path, sr=SAMPLING_RATE, mono=True)


track_id=98571 cannot be loaded


[src/libmpg123/parse.c:do_readahead():1099] warning: Cannot read next header, a one-frame stream? Duh...
/mnt/WD-shared/PJATK/PracaMagisterska/predictions_server/model_testing/data/FMA/spectrograms.py:26: UserWarning: PySoundFile failed. Trying audioread instead.
  return librosa.load(file_path, sr=SAMPLING_RATE, mono=True)


track_id=98560 cannot be loaded


[src/libmpg123/parse.c:do_readahead():1099] warning: Cannot read next header, a one-frame stream? Duh...
/mnt/WD-shared/PJATK/PracaMagisterska/predictions_server/model_testing/data/FMA/spectrograms.py:26: UserWarning: PySoundFile failed. Trying audioread instead.
  return librosa.load(file_path, sr=SAMPLING_RATE, mono=True)


track_id=143992 cannot be loaded


[src/libmpg123/layer3.c:INT123_do_layer3():1801] error: dequantization failed!


((48010, 128, 130), (48010, 12))

In [21]:
pickle_spectrograms_set(x_val, y_val, 'validation')

In [22]:
del x_val, y_val

In [23]:
x_test, y_test, _ = create_spectrograms_for_set(test_tracks.index, max_shape=max_shape)
x_test.shape, y_test.shape

  0%|          | 0/2272 [00:00<?, ?it/s]

Note: Illegal Audio-MPEG-Header 0x00000000 at offset 106439.
Note: Trying to resync...
Note: Skipped 1024 bytes in input.
[src/libmpg123/parse.c:wetwork():1365] error: Giving up resync after 1024 bytes - your stream is not nice... (maybe increasing resync limit could help).
/mnt/WD-shared/PJATK/PracaMagisterska/predictions_server/model_testing/data/FMA/spectrograms.py:26: UserWarning: PySoundFile failed. Trying audioread instead.
  return librosa.load(file_path, sr=SAMPLING_RATE, mono=True)
[src/libmpg123/parse.c:do_readahead():1099] warning: Cannot read next header, a one-frame stream? Duh...
/mnt/WD-shared/PJATK/PracaMagisterska/predictions_server/model_testing/data/FMA/spectrograms.py:26: UserWarning: PySoundFile failed. Trying audioread instead.
  return librosa.load(file_path, sr=SAMPLING_RATE, mono=True)


track_id=105247 cannot be loaded


[src/libmpg123/parse.c:do_readahead():1099] warning: Cannot read next header, a one-frame stream? Duh...
/mnt/WD-shared/PJATK/PracaMagisterska/predictions_server/model_testing/data/FMA/spectrograms.py:26: UserWarning: PySoundFile failed. Trying audioread instead.
  return librosa.load(file_path, sr=SAMPLING_RATE, mono=True)


track_id=108925 cannot be loaded


((23619, 128, 130), (23619, 12))

In [24]:
pickle_spectrograms_set(x_test, y_test, 'test')

In [ ]:
del x_test, y_test